The features.tsv file is not good so I can't use it for crocodile 5

# QC pipeline on the Nile Crocodile genome
Using h5ad files Dafna created and I will perform a QC pipeline to identify and remove low quality cells

## Imports

In [1]:
import numpy as np
import scanpy as sc
import pandas as pd
import os
import bbknn
from pathlib import Path

## Single cell settings

In [2]:
sc.settings.verbosity = 4
sc.settings.set_figure_params(80)
sc.settings._file_format_figs = 'pdf'
sc.settings.savefigs = False
use_first_n_samples = 0
full_sparse = False

C:\Users\TzachiHNB2\AppData\Local\Temp\ipykernel_24396\3661074169.py:2: FutureWarning: The specified parameters ('scanpy',) are no longer positional. Please specify them like `scanpy=80`
  sc.settings.set_figure_params(80)


## Basic QC workflow

In [3]:
adatas = []
data_names = ['control_lps','control_pic']
main_path = '/Code/data/single_cell/animals/crocodile/crocodile_5/control/'
paths =  ['\\control_lps','\\control_pic']
for path in paths:
    adata = sc.read_10x_mtx(main_path + path, var_names='gene_symbols', cache=False)  
    sc.logging.print_memory_usage()
    print(adata.shape)
    adatas.append(adata)

    reading \Code\data\single_cell\animals\crocodile\crocodile_5\control\control_lps\matrix.mtx.gz
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Memory usage: current 0.33 GB, difference +0.33 GB
(6222, 20530)
    reading \Code\data\single_cell\animals\crocodile\crocodile_5\control\control_pic\matrix.mtx.gz
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Memory usage: current 0.40 GB, difference +0.07 GB
(6214, 20530)


In [4]:
adata_control = adatas[0].concatenate(adatas[1], batch_categories=data_names, batch_key='treatment')

adata_control

c:\Python\Python310\lib\site-packages\anndata\_core\anndata.py:1763: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(


AnnData object with n_obs × n_vars = 12436 × 20530
    obs: 'treatment'
    var: 'gene_ids', 'feature_types'

In [5]:
adata_control.obs['treatment'] = 'control'
adata_control.obs

,treatment
AAACCCACAATAGGGC-1-control_lps,control
AAACCCACATAATCGC-1-control_lps,control
AAACCCAGTGCCTGCA-1-control_lps,control
AAACCCATCACGACTA-1-control_lps,control
AAACCCATCAGGAACG-1-control_lps,control
...,...
TTTGTTGCAGCGAGTA-1-control_pic,control
TTTGTTGCATGTCAGT-1-control_pic,control
TTTGTTGGTTCCTAAG-1-control_pic,control
TTTGTTGTCCAAACCA-1-control_pic,control


In [6]:
adata_control.var

,gene_ids,feature_types
TRINITY_DN903_c0_g1~~TRINITY_DN903_c0_g1_i1.p1,TRINITY_DN903_c0_g1~~TRINITY_DN903_c0_g1_i1.p1,Gene Expression
TRINITY_DN903_c0_g1~~TRINITY_DN903_c0_g1_i1.p3,TRINITY_DN903_c0_g1~~TRINITY_DN903_c0_g1_i1.p3,Gene Expression
TRINITY_DN849_c2_g2~~TRINITY_DN849_c2_g2_i1.p1,TRINITY_DN849_c2_g2~~TRINITY_DN849_c2_g2_i1.p1,Gene Expression
TRINITY_DN849_c2_g2~~TRINITY_DN849_c2_g2_i1.p2,TRINITY_DN849_c2_g2~~TRINITY_DN849_c2_g2_i1.p2,Gene Expression
TRINITY_DN684_c0_g2~~TRINITY_DN684_c0_g2_i1.p1,TRINITY_DN684_c0_g2~~TRINITY_DN684_c0_g2_i1.p1,Gene Expression
...,...,...
TRINITY_DN30_c0_g1~~TRINITY_DN30_c0_g1_i84.p9,TRINITY_DN30_c0_g1~~TRINITY_DN30_c0_g1_i84.p9,Gene Expression
TRINITY_DN2069_c0_g1~~TRINITY_DN2069_c0_g1_i90.p1,TRINITY_DN2069_c0_g1~~TRINITY_DN2069_c0_g1_i90.p1,Gene Expression
TRINITY_DN2069_c0_g1~~TRINITY_DN2069_c0_g1_i90.p2,TRINITY_DN2069_c0_g1~~TRINITY_DN2069_c0_g1_i90.p2,Gene Expression
TRINITY_DN1213_c0_g2~~TRINITY_DN1213_c0_g2_i110.p2,TRINITY_DN1213_c0_g2~~TRINITY_DN1213_c0_g2_i11...,Gene Expression


In [ ]:
adata_control.obs.index = [x.split('-control')[0] for x in adata_control.obs.index]
adata_control.obs.index

In [ ]:
adatas = [adata_control]
adata_control.obs_names_make_unique()

In [ ]:
data_names = ['lps','pic']
main_path = '/Code/data/single_cell/animals/crocodile/crocodile_5/'
paths = ['\\lps','\\pic']
for path in paths:
    adata = sc.read_10x_mtx(main_path + path, var_names='gene_symbols', cache=False)
    sc.logging.print_memory_usage()
    print(adata.shape)
    adatas.append(adata)

In [ ]:
adata_all_treatments = adata_control.concatenate([adatas[1], adatas[2]], batch_key='treatment' , batch_categories = ['control', 'lps', 'pic'])
adata_all_treatments

In [ ]:
adata_all_treatments.obs['treatment'] = adata_all_treatments.obs['treatment'].map({'control': 'control','lps': 'lps', 'pic': 'pic'})

In [ ]:
adata_all_treatments.obs

In [ ]:
adata_all_treatments.var

In [ ]:
genes = pd.read_csv('/Code/data/single_cell/animals/eggnog_croc_chic_komo_turt.csv')
genes.dropna(subset = ['crocodile gene id'], inplace=True)

genes_dict = dict(zip(genes['crocodile gene id'].astype('str'), genes.eggnog_name.astype('str'))) #change the column name to fit the animal

dict_multi, dict_uni = {}, {}
for d in genes_dict:
    if len(d.split(',')) != 1:
        dict_multi[d] = genes_dict[d]
    else:
        dict_uni[d] =  genes_dict[d]

multi_result_dict = {}

for key_str, value in dict_multi.items():
    # Convert the string key to a set
    key_set = set(key_str.strip("{}").replace("'", "").split(", "))

    # Iterate through the elements in the set and create individual keys
    for element in key_set:
        multi_result_dict[element] = value

genes_dict = {**dict_uni, **multi_result_dict}

genes_dict

In [ ]:
adata_all_treatments.obs

In [ ]:
adata_all_treatments.var

In [ ]:
# Get a Series of variable (gene) names
var_names_series = pd.Series(adata_all_treatments.var_names)

# Identify duplicate variable names
duplicate_var_names = var_names_series[var_names_series.duplicated(keep='first')]

# Iterate through duplicates and add suffixes to make them unique from the first occurrence
for duplicate_name in duplicate_var_names:
    count = 1
    indices = var_names_series[var_names_series == duplicate_name].index
    for idx in indices:
        var_names_series[idx] = f'{duplicate_name}_{count}'
        count += 1

# Update the AnnData object with the unique variable names
adata_all_treatments.var_names = var_names_series.values

adata_all_treatments.var_names

In [ ]:
sc.pl.highest_expr_genes(adata_all_treatments, n_top=20)

In [ ]:
sc.pp.filter_cells(adata_all_treatments, min_genes=100) 
sc.pp.filter_genes(adata_all_treatments, min_cells=3)
print(adata_all_treatments.shape)

In [ ]:
dict_replace = {'COX1':'MT-COX1','COX2':'MT-COX2','COX3':'MT-COX3','ND1':'MT-ND1-1','ND1':'MT-ND1-2','ND2':'MT-ND2',
                'ND3':'MT-ND3','ND4':'MT-ND4','ND5':'MT-ND5','ND6':'MT-ND6','ND4L-1':'MT-ND4L','ATP6':'MT-ATP6','ATP8':'MT-ATP8',
                'CYTB':'MT-CYTB'}
adata_all_treatments.var.rename(dict_replace, inplace=True)
# ONLY MT-ATP6, MT-ND1, MT-ND4L

In [ ]:
adata_all_treatments.var['MT'] = adata_all_treatments.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'MT'
sc.pp.calculate_qc_metrics(adata_all_treatments, qc_vars=['MT'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pp.calculate_qc_metrics(adata_all_treatments, percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.scatter(adata_all_treatments, x='total_counts', y='n_genes_by_counts', color='treatment')
sc.pl.scatter(adata_all_treatments, x='total_counts', y='pct_counts_MT', color='treatment')
sc.pl.scatter(adata_all_treatments, x='total_counts', y='n_genes', color='treatment', save=True)

In [ ]:
adata_all_treatments = adata_all_treatments[adata_all_treatments.obs.pct_counts_MT < 20] 
adata_all_treatments = adata_all_treatments[adata_all_treatments.obs.total_counts < 5000, :] # If filtering outliers (<0.1% of cells)

In [ ]:
import scrublet as scr

def scrub(adatas, adata_all_treatments, adata_all_treatments_names):  # based on raw individual samples. 
    print('Before scrublet: ', adata.shape[0])
    doub_index = []
    barcodes = []
    for data,name in zip(adatas, adata_all_treatments_names):
        data.raw = data
        sc.pp.normalize_total(data, target_sum=1e4)
        sc.pp.log1p(data)
        scrub = scr.Scrublet(data.raw.X)
        data.obs['doublet_scores'], data.obs['predicted_doublets'] = scrub.scrub_doublets()
        scrub.plot_histogram()
        print('Doublets' + name + ' :', data.obs[data.obs['doublet_scores'] > 0.25].shape[0])
        barcodes = data.obs[data.obs['doublet_scores'] < 0.25].index.to_list()
        for barcode in barcodes:
            doub_index.append(barcode + '-' + name)

    adata_all_treatments = adata_all_treatments[adata_all_treatments.obs.index.isin(doub_index)]
    print('After scrublet: ', adata_all_treatments.shape[0])
    return adata_all_treatments

In [ ]:
adata_all_treatments = scrub(adatas, adata_all_treatments, data_names)

In [ ]:
cell_cycle_genes = [x.strip() for x in open(r'/Code/data/single_cell/animals/regev_lab_cell_cycle_genes.txt')]

s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]
s_genes = [x for x in s_genes if x in adata.var_names]
g2m_genes = [x for x in g2m_genes if x in adata.var_names]

In [ ]:
adata.var_names

In [ ]:
g2m_genes

In [ ]:
matching_values = adata.var_names.isin(cell_cycle_genes)

matching_values

In [ ]:
cell_cycle_adata = adata.copy()

sc.pp.normalize_per_cell(cell_cycle_adata, counts_per_cell_after=1e4)
sc.pp.log1p(cell_cycle_adata)
sc.pp.scale(cell_cycle_adata)
sc.tl.score_genes_cell_cycle(cell_cycle_adata, s_genes=s_genes, g2m_genes=g2m_genes)
adata_cc_genes = cell_cycle_adata[:, cell_cycle_genes].copy()
sc.tl.pca(adata_cc_genes)
sc.pl.pca_scatter(adata_cc_genes, color='phase', save=True)
adata.obs['S_score'] = cell_cycle_adata.obs['S_score'].copy()
adata.obs['G2M_score'] = cell_cycle_adata.obs['G2M_score'].copy()
adata.obs['phase'] = cell_cycle_adata.obs['phase'].copy()

In [ ]:
adata

In [ ]:
adata.var

In [ ]:
# adata.write(r'/Code/data/single_cell/animals/frog/frog_2/laevis/''frog_2_after_qc.h5ad')